# Imports

In [87]:
import requests 
from bs4 import BeautifulSoup
import csv 
from selenium import webdriver
import time
import sys
import argparse
from datetime import datetime
import pandas as pd


# Scraper.PY

In [40]:
!python scraper.py -i --url "https://www.tripadvisor.com/Restaurants-g186338-London_England.html"

^C


# GithHub

In [117]:
cd "C:\Users\Cyprien\OneDrive - De Vinci\Documents\ESILV\A5\Machine_Learning_NLP\Project2\nlp-project"

C:\Users\Cyprien\OneDrive - De Vinci\Documents\ESILV\A5\Machine_Learning_NLP\Project2\nlp-project


In [ ]:
!git pull

In [ ]:
!git add .
!git commit -m "modifications"
!git push origin main

[main a4aa8d1] modifications
 4 files changed, 277 insertions(+), 1 deletion(-)
 create mode 100644 msedgedriver.exe


To https://github.com/Shins0o/nlp-project
 ! [rejected]        main -> main (fetch first)
error: failed to push some refs to 'https://github.com/Shins0o/nlp-project'
hint: Updates were rejected because the remote contains work that you do
hint: not have locally. This is usually caused by another repository pushing
hint: to the same ref. You may want to first integrate the remote changes
hint: (e.g., 'git pull ...') before pushing again.
hint: See the 'Note about fast-forwards' in 'git push --help' for details.


# Scrapping

In [125]:
# url = "https://uk.trustpilot.com/categories/restaurants_bars?location=London&trustscore=4.5&verified=true" # 4.5 stars london food
# url = "https://uk.trustpilot.com/categories/restaurants_bars?location=London&trustscore=4.0&verified=true" # +4 stars london food

url = "https://uk.trustpilot.com/categories/restaurant?sort=reviews_count" # All UK restaurants
# url = "https://uk.trustpilot.com/categories/restaurant?location=London&sort=reviews_count" # All London Restaurants
urls =[]
nextPage = True
while nextPage:
    page = requests.get(url)
    time.sleep(1)
    soup = BeautifulSoup(page.text, 'html.parser')
    restaurant_links = soup.find_all('a', {'data-business-unit-card-link': 'true'})
    for link in restaurant_links:
        href = link.get('href')
        urls.append("https://uk.trustpilot.com"+href)  
    
    # Restaurants are ordered in nnumber of reviews so we stop the loop if we find a restaurant without review : the next pages won't
    # try:
    #     soup.find('div', class_ = "styles_rating__pY5Pk")
    # except: 
    #     break 

    # Go to next page or stop the loop if at the last page
    try:
        href = str(soup.find('a',{'aria-label':"Next page"})['href'])
        url = 'https://uk.trustpilot.com/' + href
    except:
        nextPage = False

len(urls)

106

## For one restaurant

In [135]:
test = "restaurant_reviews.csv"

url = "https://uk.trustpilot.com/review/dineindulge.co.uk"
# reviews = []
# for url in urls :
nextPage = True
while nextPage:
    page = requests.get(url)
    time.sleep(0.3)
    soup = BeautifulSoup(page.text, 'html.parser')

    # Information about the restaurant
    avg_rating = soup.find('p',class_="typography_body-l__KUYFJ typography_appearance-subtle__8_H2l").text
    restaurant_name = soup.find('span', class_='typography_display-s__qOjh6 typography_appearance-default__AAY17 title_displayName__TtDDM').contents[0] # not .text to take only what's before  <!-- -->'
    categories_element = soup.find_all('a',class_="link_internal__7XN06 typography_appearance-action__9NNRY link_link__IZzHN link_underlined__OXYVM")
    categories = []
    for element in categories_element :
        category  = element.text
        if category not in categories:
            categories.append(category)
    information = soup.find('div', class_='styles_container__9nZxD customer-generated-content').text

    # Putting informations about the reviews in a csv file
    try:
        with open(test, mode='a', encoding="utf-8") as trip_data:
            data_writer = csv.writer(trip_data, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
            # Writing column headers as the first row
            data_writer.writerow(['Rating', 'Restaurant Name', 'Categories', 'Average Rating', 'Experience Date', 'Username', 'Review Title', 'Information', 'Review Text'])
            # retrieving reviews
            reviews_elements = soup.find_all('article', class_="paper_paper__1PY90 paper_outline__lwsUX card_card__lQWDv card_noPadding__D8PcU styles_reviewCard__hcAvl")
            #retrieving info within reviews
            for review in reviews_elements :
                experience_date = review.find('p', class_="typography_body-m__xgxZ_ typography_appearance-default__AAY17").text.strip().split(":")[-1].strip()
                experience_date = datetime.strptime(experience_date, '%d %B %Y')
                experience_date = experience_date.strftime("%Y-%m-%d")
                review_username = review.find('span', class_="typography_heading-xxs__QKBS8 typography_appearance-default__AAY17").text
                if review.find('img', {'alt':'Rated 5 out of 5 stars'}):
                    rating = 5
                elif review.find('img', {'alt':'Rated 4 out of 5 stars'}):
                    rating = 4
                elif review.find('img', {'alt':'Rated 3 out of 5 stars'}):
                    rating = 3
                elif review.find('img', {'alt':'Rated 2 out of 5 stars'}):
                    rating = 2
                elif review.find('img', {'alt':'Rated 1 out of 5 stars'}):
                    rating = 1
                elif review.find('img', {'alt':'Rated 0 out of 5 stars'}):
                    rating = 0
                review_title = review.find('h2', class_="typography_heading-s__f7029 typography_appearance-default__AAY17").text
                try:
                    review_text = review.find('p', class_="typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn").text
                except:
                    review_text = '' # to treat it more efficiently during NLP preprocessing
                # reviews.append([rating, experience_date, review_username, review_title, review_text])
                data_writer.writerow([rating, restaurant_name, categories, avg_rating,  experience_date, review_username, review_title, information, review_text])
    except:
        print("passed")
    try:
        href = str(soup.find('a',{'aria-label':"Next page"})['href'])
        url = 'https://uk.trustpilot.com/' + href
    except:
        nextPage = False

# reviews


ConnectionError: HTTPSConnectionPool(host='uk.trustpilot.com', port=443): Max retries exceeded with url: /review/dineindulge.co.uk (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000024731B05250>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

## For all restaurant from London

In [122]:
restaurant_london_reviews = "restaurant_london_reviews.csv"
with open(restaurant_london_reviews, mode='a', encoding="utf-8") as review_data:
    data_writer = csv.writer(review_data, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
    # Writing column headers as the first row
    data_writer.writerow(['Rating', 'Restaurant Name', 'Categories', 'Average Rating', 'Experience Date', 'Username', 'Review Title', 'Information', 'Review Text'])

for url in urls :
    nextPage = True
    while nextPage:
        page = requests.get(url)
        time.sleep(1)
        soup = BeautifulSoup(page.text, 'html.parser')

        # Information about the restaurant
        avg_rating = soup.find('p',class_="typography_body-l__KUYFJ typography_appearance-subtle__8_H2l").text
        restaurant_name = soup.find('span', class_='typography_display-s__qOjh6 typography_appearance-default__AAY17 title_displayName__TtDDM').contents[0] # not .text to take only what's before  <!-- -->'
        categories_element = soup.find_all('a',class_="link_internal__7XN06 typography_appearance-action__9NNRY link_link__IZzHN link_underlined__OXYVM")
        categories = []
        for element in categories_element :
            category  = element.text
            if category not in categories:
                categories.append(category)
        try: #not all restaurants have filled info
            information = soup.find('div', class_='styles_container__9nZxD customer-generated-content').text
        except:
            information = ''

        # Putting informations about the reviews in a csv file
        try:
            with open(restaurant_london_reviews, mode='a', encoding="utf-8") as review_data:
                data_writer = csv.writer(review_data, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
                # retrieving reviews
                reviews_elements = soup.find_all('article', class_="paper_paper__1PY90 paper_outline__lwsUX card_card__lQWDv card_noPadding__D8PcU styles_reviewCard__hcAvl")
                #retrieving info within reviews
                for review in reviews_elements :
                    experience_date = review.find('p', class_="typography_body-m__xgxZ_ typography_appearance-default__AAY17").text.strip().split(":")[-1].strip()
                    experience_date = datetime.strptime(experience_date, '%d %B %Y')
                    experience_date = experience_date.strftime("%Y-%m-%d")
                    review_username = review.find('span', class_="typography_heading-xxs__QKBS8 typography_appearance-default__AAY17").text
                    if review.find('img', {'alt':'Rated 5 out of 5 stars'}):
                        rating = 5
                    elif review.find('img', {'alt':'Rated 4 out of 5 stars'}):
                        rating = 4
                    elif review.find('img', {'alt':'Rated 3 out of 5 stars'}):
                        rating = 3
                    elif review.find('img', {'alt':'Rated 2 out of 5 stars'}):
                        rating = 2
                    elif review.find('img', {'alt':'Rated 1 out of 5 stars'}):
                        rating = 1
                    elif review.find('img', {'alt':'Rated 0 out of 5 stars'}):
                        rating = 0
                    review_title = review.find('h2', class_="typography_heading-s__f7029 typography_appearance-default__AAY17").text
                    try:
                        review_text = review.find('p', class_="typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn").text
                    except:
                        review_text = '' # to treat it more efficiently during NLP preprocessing
                    data_writer.writerow([rating, restaurant_name, categories, avg_rating,  experience_date, review_username, review_title, information, review_text])
        except:
            print("passed")
        try:
            href = str(soup.find('a',{'aria-label':"Next page"})['href'])
            url = 'https://uk.trustpilot.com/' + href
        except:
            nextPage = False



## For all UK

In [129]:
restaurant_uk_reviews = "restaurant_uk_reviews.csv"
with open(restaurant_uk_reviews, mode='a', encoding="utf-8") as review_data:
    data_writer = csv.writer(review_data, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
    # Writing column headers as the first row
    data_writer.writerow(['Rating', 'Restaurant Name', 'Categories', 'Average Rating', 'Experience Date', 'Username', 'Review Title', 'Information', 'Review Text'])

for url in urls :
    nextPage = True
    while nextPage:
        page = requests.get(url)
        time.sleep(0.3)
        soup = BeautifulSoup(page.text, 'html.parser')

        # Information about the restaurant
        try:
            avg_rating = soup.find('p',class_="typography_body-l__KUYFJ typography_appearance-subtle__8_H2l").text
        except:
            break
        restaurant_name = soup.find('span', class_='typography_display-s__qOjh6 typography_appearance-default__AAY17 title_displayName__TtDDM').contents[0] # not .text to take only what's before  <!-- -->'
        categories_element = soup.find_all('a',class_="link_internal__7XN06 typography_appearance-action__9NNRY link_link__IZzHN link_underlined__OXYVM")
        categories = []
        for element in categories_element :
            category  = element.text
            if category not in categories:
                categories.append(category)
        try: #not all restaurants have filled info
            information = soup.find('div', class_='styles_container__9nZxD customer-generated-content').text
        except:
            information = ''

        # Putting informations about the reviews in a csv file
        try:
            with open(restaurant_uk_reviews, mode='a', encoding="utf-8") as review_data:
                data_writer = csv.writer(review_data, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
                # retrieving reviews
                reviews_elements = soup.find_all('article', class_="paper_paper__1PY90 paper_outline__lwsUX card_card__lQWDv card_noPadding__D8PcU styles_reviewCard__hcAvl")
                #retrieving info within reviews
                for review in reviews_elements :
                    experience_date = review.find('p', class_="typography_body-m__xgxZ_ typography_appearance-default__AAY17").text.strip().split(":")[-1].strip()
                    experience_date = datetime.strptime(experience_date, '%d %B %Y')
                    experience_date = experience_date.strftime("%Y-%m-%d")
                    review_username = review.find('span', class_="typography_heading-xxs__QKBS8 typography_appearance-default__AAY17").text
                    if review.find('img', {'alt':'Rated 5 out of 5 stars'}):
                        rating = 5
                    elif review.find('img', {'alt':'Rated 4 out of 5 stars'}):
                        rating = 4
                    elif review.find('img', {'alt':'Rated 3 out of 5 stars'}):
                        rating = 3
                    elif review.find('img', {'alt':'Rated 2 out of 5 stars'}):
                        rating = 2
                    elif review.find('img', {'alt':'Rated 1 out of 5 stars'}):
                        rating = 1
                    elif review.find('img', {'alt':'Rated 0 out of 5 stars'}):
                        rating = 0
                    review_title = review.find('h2', class_="typography_heading-s__f7029 typography_appearance-default__AAY17").text
                    try:
                        review_text = review.find('p', class_="typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn").text
                    except:
                        review_text = '' # to treat it more efficiently during NLP preprocessing
                    data_writer.writerow([rating, restaurant_name, categories, avg_rating,  experience_date, review_username, review_title, information, review_text])
        except:
            print("passed")
        try:
            href = str(soup.find('a',{'aria-label':"Next page"})['href'])
            url = 'https://uk.trustpilot.com/' + href
        except:
            nextPage = False



## Reading csv file

### London

In [124]:
dfl = pd.read_csv('restaurant_london_reviews.csv')
dfl

,Rating,Restaurant Name,Categories,Average Rating,Experience Date,Username,Review Title,Information,Review Text
0,Rating,Restaurant Name,Categories,Average Rating,Experience Date,Username,Review Title,Information,Review Text
1,1,Franco Manca,['Restaurant'],2.7,2024-01-13,Sima Nava,Rude and unfair treatment,Franco Manca - Sourdough Pizza. The pizza is m...,"The waiter was terribly rude, I visited here o..."
2,2,Franco Manca,['Restaurant'],2.7,2024-01-13,Aidy,Appalling service from staff at Exeter branch.,Franco Manca - Sourdough Pizza. The pizza is m...,Took my girlfriend to Franco Manca Exeter bran...
3,1,Franco Manca,['Restaurant'],2.7,2024-01-10,O P,Manager was rude,Franco Manca - Sourdough Pizza. The pizza is m...,Wimbledon Branch- Manager was rude. Refused to...
4,1,Franco Manca,['Restaurant'],2.7,2023-12-23,Tommy Andrews,Awful - St Paul’s branch,Franco Manca - Sourdough Pizza. The pizza is m...,"Awful, rude and disgraceful service from a fra..."
...,...,...,...,...,...,...,...,...,...
771,Rating,Restaurant Name,Categories,Average Rating,Experience Date,Username,Review Title,Information,Review Text
772,Rating,Restaurant Name,Categories,Average Rating,Experience Date,Username,Review Title,Information,Review Text
773,Rating,Restaurant Name,Categories,Average Rating,Experience Date,Username,Review Title,Information,Review Text
774,Rating,Restaurant Name,Categories,Average Rating,Experience Date,Username,Review Title,Information,Review Text


In [92]:
dfl.dtypes

Rating             object
Restaurant Name    object
Categories         object
Average Rating     object
Experience Date    object
Username           object
Review Title       object
Information        object
Review Text        object
dtype: object

In [93]:
dfl.isnull().sum()

Rating              0
Restaurant Name     0
Categories          0
Average Rating      0
Experience Date     0
Username            0
Review Title        0
Information         0
Review Text        18
dtype: int64

### UK

In [130]:
dfuk = pd.read_csv('restaurant_uk_reviews.csv')
dfuk

,Rating,Restaurant Name,Categories,Average Rating,Experience Date,Username,Review Title,Information,Review Text
0,5,Buyagift,"['Hotel', 'Adventure Sports', 'Racetrack', 'Co...",3.8,2023-12-24,SUE CHAPMAN,Good choice and reasonable prices,Buyagift is the UK's leading provider of gift ...,The experience vouchers are easy to buy and a ...
1,5,Buyagift,"['Hotel', 'Adventure Sports', 'Racetrack', 'Co...",3.8,2023-12-30,J. Cepson,Enjoy a lovely meal with our date night voucher,Buyagift is the UK's leading provider of gift ...,We were bought a Buyagift voucher for Christma...
2,3,Buyagift,"['Hotel', 'Adventure Sports', 'Racetrack', 'Co...",3.8,2023-12-23,sue,Experience was a little disappointing,Buyagift is the UK's leading provider of gift ...,"Experience was a little disappointing, as the ..."
3,5,Buyagift,"['Hotel', 'Adventure Sports', 'Racetrack', 'Co...",3.8,2024-01-10,Mrs P,Perfect all through process 👍,Buyagift is the UK's leading provider of gift ...,Recently bought an afternoon tea voucher as a ...
4,3,Buyagift,"['Hotel', 'Adventure Sports', 'Racetrack', 'Co...",3.8,2024-01-11,Yvonne Meechan,Treatments were excellent however…,Buyagift is the UK's leading provider of gift ...,Treatments were excellent however having to pa...
...,...,...,...,...,...,...,...,...,...
12313,5,Langstone Manor Holiday Park,['Camping Shop'],3.7,2022-07-09,Liam Perry,Highly recommended,Set amongst delightful mature grounds in a she...,We had a fantastic time here recently having u...
12314,5,DickHartley,['Restaurant'],3.7,2022-06-16,John,Good to do business with.,Dick Hartley is an independent supplier of kit...,Good to do business with.Bought some Riedel gl...
12315,5,The Beachfield Hotel,['Family Hotel'],3.7,2023-05-18,Bad Experience.,"Clean ,competitive pricing,great food…",Welcome to Beachfield Hotel. We're a boutique ...,"Clean ,competitive pricing,great food and staf..."
12316,5,Crofton and Park,['Jewellery Repair Service'],3.7,2023-10-25,Poppy Khaisee-Headley,Excellent and professional service,"A premium,personal concierge service.\r\n\r\nA...",Excellent and professional service. I had a di...


In [132]:
value_counts = dfuk['Restaurant Name'].value_counts()
result_df = value_counts.reset_index()
result_df.columns = ['name', 'count']
print(result_df)

                            name  count
0                       Buyagift   8280
1               Zizzi Ristorante    571
2                   Franco Manca    270
3                    Dineindulge    260
4                  Barmans.co.uk    180
..                           ...    ...
79  Langstone Manor Holiday Park      1
80                   DickHartley      1
81          The Beachfield Hotel      1
82              Crofton and Park      1
83           Haute Dolci Watford      1

[84 rows x 2 columns]


In [ ]:
dfuk.dtypes

Rating             object
Restaurant Name    object
Categories         object
Average Rating     object
Experience Date    object
Username           object
Review Title       object
Information        object
Review Text        object
dtype: object

In [131]:
dfuk.isnull().sum()

Rating                0
Restaurant Name       0
Categories            0
Average Rating        0
Experience Date       0
Username              0
Review Title          0
Information         349
Review Text        1311
dtype: int64